Weather - (snow, rain, intense conditions)  
Taxi1 - (groupby credit card and only take fare + tip)  
Taxi2 - only take datetime and duration  

Questions:  
- Does the day of week (weekend/normal day) affect trip duration
- logplot for output, dataframe table for analysis to talk about

- Does weather affect trip duration (due to potential delays and road closures)

- Do the delays caused by weather potentially increase profit?
- First of all, prove that the correlation between trip duration and fare is positive linear
- Second, show that the correlation between tips and fare is positive
- IF it is positive, then we can say that potentially weather impact may lead to increase durations (albeit less frequency)

Subquestions:  
- Why are airports short in freq/duration? (snow, shuttle bus)
- Why do some zones have longer durations than others during months that cannot be explained by the data [choropleth by months] (road works)

As long as assumptions justified and sources provided, all is good


In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

"""
gf = gpd.GeoDataFrame(pd.merge(df, sf, left_on=['PULocationID'], right_on=['LocationID']))
"""

"\ngf = gpd.GeoDataFrame(pd.merge(df, sf, left_on=['PULocationID'], right_on=['LocationID']))\n"

In [2]:
num_to_dow = {
    1: 'Monday',
    2: 'Tuesday',
    3: 'Wednesday',
    4: 'Thursday',
    5: 'Friday',
    6: 'Saturday',
    0: 'Sunday'
}

# Taxi 2 Data

- Does the day of week (weekend/normal day) affect trip duration
- logplot for output, dataframe table for analysis to talk about

In [3]:
df = pd.read_feather('./data/v2/2018taxi_durations.feather').drop('index', axis=1)

C:\Users\akira\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [4]:
df.head()

,PUdatetime,DOdatetime,PULocationID,trip_duration,trip_distance
0,2018-01-01 00:21:05,2018-01-01 00:24:23,41,3,0.5
1,2018-01-01 00:44:55,2018-01-01 01:03:05,239,18,2.7
2,2018-01-01 00:08:26,2018-01-01 00:14:21,262,5,0.8
3,2018-01-01 00:20:22,2018-01-01 00:52:51,140,32,10.2
4,2018-01-01 00:09:18,2018-01-01 00:27:06,246,17,2.5


In [7]:
def workday(x):
    if x in ['Saturday', 'Sunday']:
        return 0
    return 1

In [8]:
df['Day of Week'] = df['PUdatetime'].dt.dayofweek
df.drop('DOdatetime', axis=1, inplace=True)

df['Day of Week'] = df['Day of Week'].apply(lambda x: num_to_dow[x])
df['Is_WorkDay'] = df['Day of Week'].apply(lambda x: workday(x))

df['log_duration'] = np.log(df['trip_duration'])

In [9]:
df.head()

,PUdatetime,PULocationID,trip_duration,trip_distance,Day of Week,Is_WorkDay,log_duration
0,2018-01-01 00:21:05,41,3,0.5,Sunday,0,1.098612
1,2018-01-01 00:44:55,239,18,2.7,Sunday,0,2.890372
2,2018-01-01 00:08:26,262,5,0.8,Sunday,0,1.609438
3,2018-01-01 00:20:22,140,32,10.2,Sunday,0,3.465736
4,2018-01-01 00:09:18,246,17,2.5,Sunday,0,2.833213


In [ ]:
# This dataset is used to see duration/distance/logduration by day of week, and also can be used to
# plot violin plot of workday vs non-workday changing datetime to months
df.reset_index().to_feather('./data/v2/dow_durations.feather')

In [10]:
agg_df_mean = df.groupby('Is_WorkDay')[['trip_duration',
                                   'trip_distance','log_duration']].mean().reset_index()
agg_df_mean.columns = [i + '_mean' if i != 'Is_WorkDay' else i for i in agg_df_mean.columns]

agg_df_std = df.groupby('Is_WorkDay')[['trip_duration',
                                   'trip_distance','log_duration']].std().reset_index()
agg_df_std.columns = [i + '_std' if i != 'Is_WorkDay' else i for i in agg_df_std.columns]

agg_df = pd.merge(agg_df_mean, agg_df_std, on='Is_WorkDay')

In [11]:
agg_df.head()

,Is_WorkDay,trip_duration_mean,trip_distance_mean,log_duration_mean,trip_duration_std,trip_distance_std,log_duration_std
0,0,16.317133,3.151116,2.279878,68.105006,4.146053,0.832793
1,1,17.425457,2.895147,2.375643,66.466553,22.211267,0.829152


In [ ]:
# This dataset is good to see weeknds vs weekdays in a plot
agg_df.to_csv('./data/v2/by_workday_agg.csv')

- Does weather affect trip duration (due to potential delays and road closures)

In [5]:
weather = pd.read_csv('./data/weather.csv', index_col=0)
weather['datetime'] = pd.to_datetime(weather['datetime'])

weather.head()

,datetime,tmpc,heavy_snow,snow,heavy_rain,rain
0,2017-12-31 00:00:00,-6.000000,0,0,0,0
1,2017-12-31 01:00:00,-6.110000,0,0,0,0
2,2017-12-31 02:00:00,-6.387500,0,0,0,0
3,2017-12-31 03:00:00,-6.683333,0,0,0,0
4,2017-12-31 04:00:00,-7.036667,0,0,0,0


In [6]:
# Round to the closest hour and add months column
df['datetime'] = df['PUdatetime'].dt.round('H')
df['month'] = df['PUdatetime'].dt.month

weather_df = pd.merge(df, weather, on='datetime', how='left')

In [7]:
weather_df['count'] = 1

In [8]:
weather_df.head()

,PUdatetime,DOdatetime,PULocationID,trip_duration,trip_distance,datetime,month,tmpc,heavy_snow,snow,heavy_rain,rain,count
0,2018-01-01 00:21:05,2018-01-01 00:24:23,41,3,0.5,2018-01-01 00:00:00,1,-12.056667,0.0,0.0,0.0,0.0,1
1,2018-01-01 00:44:55,2018-01-01 01:03:05,239,18,2.7,2018-01-01 01:00:00,1,-11.917500,0.0,0.0,0.0,0.0,1
2,2018-01-01 00:08:26,2018-01-01 00:14:21,262,5,0.8,2018-01-01 00:00:00,1,-12.056667,0.0,0.0,0.0,0.0,1
3,2018-01-01 00:20:22,2018-01-01 00:52:51,140,32,10.2,2018-01-01 00:00:00,1,-12.056667,0.0,0.0,0.0,0.0,1
4,2018-01-01 00:09:18,2018-01-01 00:27:06,246,17,2.5,2018-01-01 00:00:00,1,-12.056667,0.0,0.0,0.0,0.0,1


In [ ]:
all_snow = weather_df.groupby('month')[['heavy_snow','snow']].sum().reset_index()
all_rain = weather_df.groupby('month')[['heavy_rain','rain']].sum().reset_index()

In [ ]:
weather_duration = weather_df.groupby('month')[['trip_duration',
                                        'log_duration', 'trip_distance', 'count','tmpc']].agg({'trip_duration': 'mean',
                                                                                       'log_duration': 'mean',
                                                                                       'trip_distance': 'mean',
                                                                                        'tmpc': 'mean',
                                                                                       'count': 'sum'}).reset_index()

In [ ]:
weather_duration = pd.merge(weather_duration, pd.merge(all_snow, all_rain, on='month'), on='month')

In [ ]:
weather_duration.head()

In [ ]:
# This dataset is an aggregation of ALL attributes by MONTH
# Includes TOTAL SUM / FREQ of trips made, how "snowy" or "rainy" the month was as well
weather_duration.to_csv('./data/v2/by_month.csv')

In [22]:
weather_zone = weather_df.groupby(['PULocationID','month']).agg({'trip_duration': 'mean',
                                                              'trip_distance': 'mean',
                                                              'tmpc': 'max',
                                                              'heavy_snow': 'sum',
                                                              'snow': 'sum',
                                                              'heavy_rain': 'sum',
                                                              'rain': 'sum',
                                                              'count': 'sum'}).reset_index()

In [24]:
weather_zone.to_csv('./data/v2/by_zone.csv')

In [12]:
weather_df.head()

,PUdatetime,DOdatetime,PULocationID,trip_duration,trip_distance,datetime,month,tmpc,heavy_snow,snow,heavy_rain,rain,count
0,2018-01-01 00:21:05,2018-01-01 00:24:23,41,3,0.5,2018-01-01 00:00:00,1,-12.056667,0.0,0.0,0.0,0.0,1
1,2018-01-01 00:44:55,2018-01-01 01:03:05,239,18,2.7,2018-01-01 01:00:00,1,-11.917500,0.0,0.0,0.0,0.0,1
2,2018-01-01 00:08:26,2018-01-01 00:14:21,262,5,0.8,2018-01-01 00:00:00,1,-12.056667,0.0,0.0,0.0,0.0,1
3,2018-01-01 00:20:22,2018-01-01 00:52:51,140,32,10.2,2018-01-01 00:00:00,1,-12.056667,0.0,0.0,0.0,0.0,1
4,2018-01-01 00:09:18,2018-01-01 00:27:06,246,17,2.5,2018-01-01 00:00:00,1,-12.056667,0.0,0.0,0.0,0.0,1


# Taxi 2 Data

_Based off months_
- Do the delays caused by weather potentially increase profit?
    - First of all, prove that the correlation between trip duration and fare is positive linear
    - Second, show that the correlation between tips and fare is positive
    - IF it is positive, then we can say that potentially weather impact may lead to increase durations (albeit less frequency)


In [2]:
df1 = pd.read_feather('./data/v2/2018taxi_fares.feather').drop('index',axis=1)
df1['month'] = df1['PUdatetime'].dt.month

C:\Users\akira\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [12]:
first_proof = pd.merge(df[['trip_duration', 
                           'month']].groupby('month').mean().reset_index(), 
                       df1[['fare_amount', 
                            'month']].groupby('month').mean().reset_index(), on='month')

In [17]:
first_proof.drop('month',axis=1).corr()s

,trip_duration,fare_amount
trip_duration,1.000000,0.889485
fare_amount,0.889485,1.000000


In [20]:
df1[['fare_amount','tip_amount']].corr()

,fare_amount,tip_amount
fare_amount,1.000000,0.756705
tip_amount,0.756705,1.000000


        - From above, we can see that obviously duration and amount have a strong positive correlation
        - Furthermore, there is also a strong positive correlation that those who had a larger fare, 
                tipped more as well (only considering card payments)

# Subquestions
- Why are airports short in freq/duration

In [21]:
df.head()

,PUdatetime,PULocationID,trip_duration,trip_distance,Day of Week,Is_WorkDay,log_duration,datetime,month
0,2018-01-01 00:21:05,41,3,0.5,Sunday,0,1.098612,2018-01-01 00:00:00,1
1,2018-01-01 00:44:55,239,18,2.7,Sunday,0,2.890372,2018-01-01 01:00:00,1
2,2018-01-01 00:08:26,262,5,0.8,Sunday,0,1.609438,2018-01-01 00:00:00,1
3,2018-01-01 00:20:22,140,32,10.2,Sunday,0,3.465736,2018-01-01 00:00:00,1
4,2018-01-01 00:09:18,246,17,2.5,Sunday,0,2.833213,2018-01-01 00:00:00,1


In [27]:
df1.head()

,PUdatetime,DOdatetime,PULocationID,fare_amount,tip_amount,month
0,2018-01-01 00:08:26,2018-01-01 00:14:21,262,6.0,1.00,1
1,2018-01-01 00:09:18,2018-01-01 00:27:06,246,12.5,2.75,1
2,2018-01-01 00:38:08,2018-01-01 00:48:24,50,9.0,2.05,1
3,2018-01-01 00:49:29,2018-01-01 00:51:53,239,4.0,1.00,1
4,2018-01-01 00:56:38,2018-01-01 01:01:05,238,5.5,1.70,1


In [36]:
left = df.groupby('PULocationID')[['trip_duration','log_duration','trip_distance']].mean().reset_index()
right = df1.groupby('PULocationID')[['fare_amount','tip_amount']].mean().reset_index()
freq = df.groupby('PULocationID')['trip_duration'].count().reset_index().rename({'trip_duration': 'freq'},axis=1)

In [37]:
zone_agg = pd.merge(pd.merge(left, right, on='PULocationID'), freq, on='PULocationID')

In [38]:
zone_agg.head()

,PULocationID,trip_duration,log_duration,trip_distance,fare_amount,tip_amount,freq
0,1,29.272941,1.981902,9.654553,72.405204,13.829650,850
1,2,35.042254,3.200203,12.238028,34.565217,7.570435,71
2,3,29.177162,2.985264,7.837792,28.430118,0.994877,1191
3,4,16.102240,2.323058,2.659884,12.480253,2.258586,210729
4,5,79.475248,4.035608,25.395149,78.415789,0.502737,101


In [39]:
# This dataset is an aggregation of ALL attributes by ZONES
# Useful for plotting choropleth for duration/logduration/distance/fare/tip/freq
zone_agg.to_csv('./data/v2/zone_agg.csv')

In [40]:
weather.head()

,datetime,tmpc,heavy_snow,snow,heavy_rain,rain
0,2017-12-31 00:00:00,-6.000000,0,0,0,0
1,2017-12-31 01:00:00,-6.110000,0,0,0,0
2,2017-12-31 02:00:00,-6.387500,0,0,0,0
3,2017-12-31 03:00:00,-6.683333,0,0,0,0
4,2017-12-31 04:00:00,-7.036667,0,0,0,0


In [41]:
df.head()

,PUdatetime,PULocationID,trip_duration,trip_distance,Day of Week,Is_WorkDay,log_duration,datetime,month
0,2018-01-01 00:21:05,41,3,0.5,Sunday,0,1.098612,2018-01-01 00:00:00,1
1,2018-01-01 00:44:55,239,18,2.7,Sunday,0,2.890372,2018-01-01 01:00:00,1
2,2018-01-01 00:08:26,262,5,0.8,Sunday,0,1.609438,2018-01-01 00:00:00,1
3,2018-01-01 00:20:22,140,32,10.2,Sunday,0,3.465736,2018-01-01 00:00:00,1
4,2018-01-01 00:09:18,246,17,2.5,Sunday,0,2.833213,2018-01-01 00:00:00,1


In [42]:
df1.head()

,PUdatetime,DOdatetime,PULocationID,fare_amount,tip_amount,month
0,2018-01-01 00:08:26,2018-01-01 00:14:21,262,6.0,1.00,1
1,2018-01-01 00:09:18,2018-01-01 00:27:06,246,12.5,2.75,1
2,2018-01-01 00:38:08,2018-01-01 00:48:24,50,9.0,2.05,1
3,2018-01-01 00:49:29,2018-01-01 00:51:53,239,4.0,1.00,1
4,2018-01-01 00:56:38,2018-01-01 01:01:05,238,5.5,1.70,1
